In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from implementations import *
from train import *

%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
DATA_TEST_PATH = '../data/test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

## Do your thing crazy machine learning thing here :) ...

In [4]:
tX, tX_test = replace_data(tX, replace=-999, by=np.NaN), replace_data(tX_test, replace= -999, by=np.NaN)

y = replace_data(y, -1, 0)

In [5]:
# initialize the range of hyperparameters
degrees = np.arange(2, 6)
lambdas = np.logspace(-4, 0, 30)
gamma = 0.005

# initialize the number of k-fold
k_fold = 4

# initialize the maximum number of iterations for updating the weights when using gradient descent
max_iters = 1000

In [ ]:
from train_least_squares_GD import *

best_weights, best_degree = train_models_least_squares_GD(y, tX, degrees, gamma, max_iters, k_fold)


/Users/olivier/Documents/GitHub/EPFL-Machine-Learning-Higgs-2019/scripts/gradient_descent.py:7: RuntimeWarning: invalid value encountered in matmul
  e = y - tx @ w
/Users/olivier/Documents/GitHub/EPFL-Machine-Learning-Higgs-2019/scripts/gradient_descent.py:9: RuntimeWarning: invalid value encountered in matmul
  return (-1 / n) * tx.T @ e
/Users/olivier/Documents/GitHub/EPFL-Machine-Learning-Higgs-2019/scripts/proj1_helpers.py:30: RuntimeWarning: invalid value encountered in less_equal
  y_pred[np.where(y_pred <= 0)] = -1
/Users/olivier/Documents/GitHub/EPFL-Machine-Learning-Higgs-2019/scripts/proj1_helpers.py:31: RuntimeWarning: invalid value encountered in greater
  y_pred[np.where(y_pred > 0)] = 1
/Users/olivier/Documents/GitHub/EPFL-Machine-Learning-Higgs-2019/scripts/gradient_descent.py:7: RuntimeWarning: overflow encountered in matmul
  e = y - tx @ w
/Users/olivier/Documents/GitHub/EPFL-Machine-Learning-Higgs-2019/scripts/gradient_descent.py:9: RuntimeWarning: overflow encounte

In [ ]:
best_degree

In [ ]:
#best_lambda

In [ ]:
#best_gamma

In [ ]:
best_weights

## Generate predictions and save ouput in csv format for submission:

In [ ]:
# get the indices of the subsets in the training and the test datasets
indices_train_group = group_indices(tX)
indices_test_group = group_indices(tX_test)

In [ ]:
# initialize the prediction array
y_pred = np.zeros(tX_test.shape[0])

y_pred = np.zeros(tX_test.shape[0])
for i, indice_test_group in enumerate(indices_test_group):
    y_pred_subset = predict_labels(best_weights[i], build_poly(drop_na_columns(tX_test[indice_test_group]), best_degree[i]))
    y_pred[indice_test_group] = y_pred_subset
    
    
y_pred

In [ ]:
### TEST ###

for i, indice_test_group in enumerate(indices_test_group):
    
    # for standardizing the test subset, we need the data of both train and test subsets
    tx_subset = tX[indices_train_group[i]]
    tx_test_subset = tX_test[indice_test_group]
    
    # get the standardized test subset
    _, standardized_tx_test_subset = preprocess_data(tx_subset, tx_test_subset)
    
    # predict the labels
    y_pred_subset = predict_labels(best_weights[i], build_poly(standardized_tx_test_subset, best_degree[i]))
    y_pred[indice_test_group] = y_pred_subset
    
    
y_pred

In [ ]:
OUTPUT_PATH = '../data/sample-submission_least_squares_GD.csv'
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

# Test on outlier detection

In [ ]:
#### Test on outlier detection

a = drop_na_columns(tX[indices_train_group[0]])[:, 0]
a

In [ ]:
b = standardize(a)[0]
b

In [ ]:
print(b.sort)

In [ ]:
import pandas as pd

In [ ]:
### convert each subset to pandas



data_sub1 = standardize(drop_na_columns(tX[indices_train_group[0]]))[0]
data_sub2 = standardize(drop_na_columns(tX[indices_train_group[1]]))[0]
data_sub3 = standardize(drop_na_columns(tX[indices_train_group[2]]))[0]
data_sub4 = standardize(drop_na_columns(tX[indices_train_group[3]]))[0]
data_sub5 = standardize(drop_na_columns(tX[indices_train_group[4]]))[0]
data_sub6 = standardize(drop_na_columns(tX[indices_train_group[5]]))[0]

df1 = pd.DataFrame(data=data_sub1[1:,1:])
df2 = pd.DataFrame(data=data_sub2[1:,1:])
df3 = pd.DataFrame(data=data_sub3[1:,1:])
df4 = pd.DataFrame(data=data_sub4[1:,1:])
df5 = pd.DataFrame(data=data_sub5[1:,1:])
df6 = pd.DataFrame(data=data_sub6[1:,1:])

In [ ]:
df_stats1 = df1.describe()
df_stats2 = df2.describe()
df_stats3 = df3.describe()
df_stats4 = df4.describe()
df_stats5 = df5.describe()
df_stats6 = df6.describe()

In [ ]:
df_stats1

In [ ]:
a = pd.Series([])
b = pd.Series([2, 3])
a.append(b)

In [ ]:
outliers = pd.Series([])


for i in range(df1.shape[1]):
    stat = df1.describe()[i]
    mean = stat['mean']
    std = stat['std']
    column = df1[i]
    new_outliers = column[(column > (mean + 3*std))]
    
    outliers = outliers.append(new_outliers)

bad_outliers = outliers.groupby(level=0).count()

bad_outliers[bad_outliers >= 3]


In [ ]:
df1.describe()[0]['mean']

In [ ]:
first_column = df1[0]
#pd.concat(df[(first_column > (mean_first_column + 3*std_first_column))], df[(first_column < (mean_first_column - 3*std_first_column))])

In [ ]:
first_column[(first_column > (mean_first_column + 3*std_first_column))]

In [ ]:
df1[(first_column < (mean_first_column - 3*std_first_column))]

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(17,17))

df1.boxplot(ax=axes[0,0])
df2.boxplot(ax=axes[0,1])
df3.boxplot(ax=axes[1,0])
df4.boxplot(ax=axes[1,1])
df5.boxplot(ax=axes[2,0])
df6.boxplot(ax=axes[2,1])
